### Authentification & initial params

In [5]:
! pip install forest_guard

You should consider upgrading via the '/home/arthur/.pyenv/versions/3.8.6/envs/projetfinal/bin/python -m pip install --upgrade pip' command.


In [ ]:
#TO BE DONE IN GOOGLE COLAB

# !git clone https://github.com/TheChwal/forest_guard.git

# %cd forest_guard/

# !pip install .

In [1]:
import ee 
import folium
from datetime import datetime as dt
from IPython.display import Image
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from google.cloud import storage
import time
import numpy as np
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import metrics
from tensorflow.python.keras import optimizers

In [ ]:
#TO BE DONE IN GOOGLE COLAB

# from google.colab import auth

# auth.authenticate_user()

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AY0e-g49zA8legl6RjwJHjhg7k4GLntVob2c_Lcy7ziSujhnGs_KdtbBnrg

Successfully saved authorization token.


In [4]:
import os
from google.cloud import storage

BUCKET = 'forest_guard_bucket'  # ⚠️ replace with your BUCKET NAME
FOLDER = 'Arthur_prediction_beijing'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

OPTICAL_BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
THERMAL_BANDS = ['B10', 'B11']
BANDS = OPTICAL_BANDS + THERMAL_BANDS
RESPONSE = 'fnf'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]

COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Specify model training parameters.
#BATCH_SIZE = 16
# EPOCHS = 10
#BUFFER_SIZE = 2000
# OPTIMIZER = 'SGD'
# LOSS = 'MeanSquaredError'
# METRICS = ['RootMeanSquaredError']

In [6]:
from forest_guard.datas import get_image_landsat

image = get_image_landsat('2015-01-01', '2017-12-31')

In [7]:
from forest_guard.datas import get_jaxa
jaxa = get_jaxa('2017-01-01', '2017-12-31')

### Download predict image

In [8]:
#Predict parameters

user_folder = 'users/bastidearthur' # replace 'username' by your GEE username (https://code.earthengine.google.com/ => Assets)
predict_image = 'FCNN_beijing_adam_binary_crossentropy' #precise zone + model params
predict_kernel_buffer = [128, 128]
predict_region = ee.Geometry.Polygon(
        [[[115.9662455210937, 40.121362012835235],
          [115.9662455210937, 39.64293313749715],
          [117.01818643906245, 39.64293313749715],
          [117.01818643906245, 40.121362012835235]]], None, False)

In [9]:
from forest_guard.predict import get_image

ModuleNotFoundError: No module named 'forest_guard.predict'

In [ ]:
# Run the download
get_predict_image(predict_image, predict_kernel_buffer, predict_region)

In [21]:
def get_predict_image(predict_image, predict_kernel_buffer, predict_region):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = image.select(BANDS),
    description = predict_image,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + predict_image,
    region = predict_region.getInfo()['coordinates'],
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': predict_kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [24]:
get_predict_image(predict_image, predict_kernel_buffer, predict_region)

Running image export to Cloud Storage...
Image export completed.


### Predict target 

In [ ]:
# Select the model from Cloud Storage to use for prediction
MODEL_SAVE = 'gs://' + 'forest_guard_bucket' + '/' + 'models/forest_guard/model_large_dataset_adam_binarycrossentropy'
model_pretrained = tf.keras.models.load_model(MODEL_SAVE)

In [ ]:
from forest_guard.predict import predict_image_target

In [25]:
def predict_image_target(predict_image, user_folder, predict_kernel_buffer, predict_region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if predict_image in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(predict_kernel_buffer[0] / 2)
  y_buffer = int(predict_kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + predict_kernel_buffer[0],
      KERNEL_SHAPE[1] + predict_kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = model_pretrained.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  featured_predictions = np.where(predictions>0.5,1,predictions)
  featured_predictions = np.where(featured_predictions<0.5,0,featured_predictions)

  print('Writing predictions...')
  out_image_file = 'gs://' + BUCKET + '/' + FOLDER + '/' + predict_image + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in featured_predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'fnf': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + predict_image
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

In [26]:
# Run the prediction.
predict_image_target(predict_image, user_folder, predict_kernel_buffer, predict_region)

Looking for TFRecord files...
['gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropy00000.tfrecord.gz',
 'gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropy00001.tfrecord.gz',
 'gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropy00002.tfrecord.gz',
 'gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropy00003.tfrecord.gz',
 'gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropy00004.tfrecord.gz']
gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropymixer.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 15,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            115.9662149728414,
                                            0.0,
                                      

### Display prediction on map

In [9]:
palette = ['006600',
           'FEFF99',
            #'0000FF'
          ]

In [11]:
# Display predicted target on a map

prediction_image = ee.Image(user_folder + '/' + predict_image)
mapid = prediction_image.getMapId({'bands': ['fnf'],
                       'min': 0, 
                       'max': 2, 
                      'palette':palette
                      })
map = folium.Map(location=[39.898, 116.5097])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='jaxa fnf',
    color=palette,
    opacity = 1
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [13]:
# Display predicted target on a map

prediction_image = ee.Image(user_folder + '/' + 'FCNN_demo_beijing_384_')
mapid = prediction_image.getMapId({'bands': ['fnf'],
                       'min': 0, 
                       'max': 2, 
                      'palette':palette
                      })
map = folium.Map(location=[39.898, 116.5097])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='jaxa fnf',
    color=palette,
    opacity = 1
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [12]:
# Display y_true to compare

mapid = jaxa.getMapId({'bands': ['fnf'],
                       'min': 2, 
                       'max': 0, 
                      'palette':palette
                      })
map = folium.Map(location=[39.898, 116.5097])


folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='jaxa fnf',
    color=palette,
    opacity = 1
  ).add_to(map)
map.add_child(folium.LayerControl())
map

### Compare specific part of test image

In [14]:
def get_image_data(predict_image, user_folder, predict_kernel_buffer, predict_region):
    """Perform inference on exported imagery, upload to Earth Engine.
    """
    print('Looking for TFRecord files...')
    # Get a list of all the files in the output bucket.
    filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}
    # Get only the files generated by the image export.
    exportFilesList = [s for s in filesList if predict_image in s]
    # Get the list of image files and the JSON mixer file.
    imageFilesList = []
    jsonFile = None
    for f in exportFilesList:
        if f.endswith('.tfrecord.gz'):
            imageFilesList.append(f)
        elif f.endswith('.json'):
            jsonFile = f
    # Make sure the files are in the right order.
    imageFilesList.sort()
    from pprint import pprint
    pprint(imageFilesList)
    print(jsonFile)
    import json
    # Load the contents of the mixer file to a JSON object.
    jsonText = !gsutil cat {jsonFile}
    # Get a single string w/ newlines from the IPython.utils.text.SList
    mixer = json.loads(jsonText.nlstr)
    pprint(mixer)
    patches = mixer['totalPatches']
    # Get set up for prediction.
    x_buffer = int(predict_kernel_buffer[0] / 2)
    y_buffer = int(predict_kernel_buffer[1] / 2)
    buffered_shape = [
        KERNEL_SHAPE[0] + predict_kernel_buffer[0],
        KERNEL_SHAPE[1] + predict_kernel_buffer[1]]
    imageColumns = [
      tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
        for k in BANDS
    ]
    imageFeaturesDict = dict(zip(BANDS, imageColumns))
    def parse_image(example_proto):
        return tf.io.parse_single_example(example_proto, imageFeaturesDict)
    def toTupleImage(inputs):
        inputsList = [inputs.get(key) for key in BANDS]
        stacked = tf.stack(inputsList, axis=0)
        stacked = tf.transpose(stacked, [1, 2, 0])
        return stacked
     # Create a dataset from the TFRecord file(s) in Cloud Storage.
    imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
    imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
    imageDataset = imageDataset.map(toTupleImage).batch(1)
    
    return imageDataset

In [15]:
image_data = get_image_data(predict_image, user_folder, predict_kernel_buffer, predict_region)

Looking for TFRecord files...
['gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropy00000.tfrecord.gz',
 'gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropy00001.tfrecord.gz',
 'gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropy00002.tfrecord.gz',
 'gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropy00003.tfrecord.gz',
 'gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropy00004.tfrecord.gz']
gs://forest_guard_bucket/Arthur_prediction_beijing/FCNN_beijing_adam_binary_crossentropymixer.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 15,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            115.9662149728414,
                                            0.0,
                                      

In [16]:
predictions = model_pretrained.predict(image_data, verbose=1)

NameError: name 'model_pretrained' is not defined

In [30]:
featured_predictions = np.where(predictions>0.5,1,predictions)
featured_predictions = np.where(predictions<0.5,0,featured_predictions)

In [31]:
stacked_image_data = np.stack(list(image_data))

In [41]:
from ipywidgets import interact

@interact(image_number=np.arange(0, 91, 1))

def plot_y_pred(image_number):
    fig = plt.figure(figsize=(15,7))
    
    plt.subplot(1, 2, 1)
    plt.imshow(np.flip(stacked_image_data[image_number,0,:,:,1:4], axis=2)*5)
    
    plt.subplot(1, 2, 2)
    plt.imshow(featured_predictions[image_number,:,:,0])
    
plot_y_pred(image_number)

interactive(children=(Dropdown(description='image_number', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, …

NameError: name 'image_number' is not defined